In [1]:
import cv2  # 导入OpenCV库
import numpy as np  # 导入NumPy库

class Sift:
    def stitch(self, images, ratio=0.75, reprojThresh=4.0, showMatches=False):
        # 拼接图像
        (right_image, left_image) = images

        # 检测并描述关键点和特征
        (left_k, left_features) = self.detectAndDescribe(left_image)
        (right_k, right_features) = self.detectAndDescribe(right_image)

        # 匹配关键点
        M = self.matchKeypoints(left_k, right_k, left_features, right_features, ratio, reprojThresh)

        # 若未匹配到足够关键点，则返回空
        if M is None:
            return None

        # 应用透视变换
        (matches, H, status) = M
        result = cv2.warpPerspective(left_image, H, (left_image.shape[1] + right_image.shape[1], left_image.shape[0]))

        # 混合重叠部分
        for r in range(result.shape[0]):
            left = 0
            for c in range(result.shape[1] // 2):
                if result[r, c].any():  # 如果有重叠
                    if left == 0:
                        left = c
                    alpha = (c - left) / (result.shape[1] // 2 - left)
                    result[r, c] = right_image[r, c] * (1 - alpha) + result[r, c] * alpha
                else:
                    result[r, c] = right_image[r, c]

        # 显示匹配结果
        if showMatches:
            vis = self.drawMatches(left_image, right_image, left_k, right_k, matches, status)
            return (result, vis)

        return result

    # 检测并描述关键点和特征
    def detectAndDescribe(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # 转换为灰度图像
        descriptor = cv2.SIFT_create()  # 创建SIFT对象
        (kps, features) = descriptor.detectAndCompute(gray, None)  # 检测关键点并计算特征
        kps = np.float32([kp.pt for kp in kps])  # 提取关键点位置
        return kps, features

    # 匹配关键点
    def matchKeypoints(self, kpsA, kpsB, featuresA, featuresB, ratio, reprojThresh):
        matcher = cv2.DescriptorMatcher_create("BruteForce")  # 创建描述符匹配器

        # 使用KNN匹配特征点
        rawMatches = matcher.knnMatch(featuresA, featuresB, 2)

        matches = []
        for m in rawMatches:
            # 当第一个匹配的距离小于第二个的一定比例时，保留该匹配
            if len(m) == 2 and m[0].distance < m[1].distance * ratio:
                matches.append((m[0].trainIdx, m[0].queryIdx))

        if len(matches) > 4:
            ptsA = np.float32([kpsA[i] for (_, i) in matches])
            ptsB = np.float32([kpsB[i] for (i, _) in matches])

            # 使用RANSAC算法计算单应性矩阵
            (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, reprojThresh)

            return (matches, H, status)

        return None

    # 绘制匹配结果
    def drawMatches(self, imageA, imageB, kpsA, kpsB, matches, status):
        (hA, wA) = imageA.shape[:2]
        (hB, wB) = imageB.shape[:2]
        vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
        vis[0:hA, 0:wA] = imageA
        vis[0:hB, wA:] = imageB

        for ((trainIdx, queryIdx), s) in zip(matches, status):
            if s == 1:
                ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
                ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
                cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

        return vis

if __name__ == '__main__':
    # 读取图像
    imageA = cv2.imread("left3.jpg")
    imageB = cv2.imread("right3.jpg")

    # 创建SIFT对象并拼接图像
    sift = Sift()
    (result, vis) = sift.stitch([imageA, imageB], showMatches=True)

    # 保存拼接结果
    cv2.imwrite("result3.png", result)
    print("拼接完成")


拼接完成
